In [27]:
import pandas as pd

In [28]:
df = pd.read_csv("spotify_millsongdata.csv")


In [29]:
df.head(5)


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [31]:
df.tail(5)


,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [32]:
df.shape


(57650, 4)

In [33]:
df.isnull().sum()


artist    0
song      0
link      0
text      0
dtype: int64

In [34]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)


In [35]:
df.head(10)


,artist,song,text
0,Venom,Beauty And The Beast,He watches witches weaving wonders \r\nSeeing...
1,Old 97's,Niteclub,Eighteen-hundred miles from this old niteclub ...
2,"Harry Connick, Jr.",Parade Of The Wooden Soldiers,The toy shop door is locked up tight \r\nAnd ...
3,Moody Blues,Words You Say,Words you say to me \r\nI listen in sympathy ...
4,Glee,Don't Stop,Don't stop! Don't stop! \r\n \r\nIf you wake...
5,Youngbloodz,6 To 14 In 12,"hook \r\n6 to 14 in 12, y'all motherfuckers s..."
6,Depeche Mode,A Question Of Lust,Fragile \r\nLike a baby in your arms \r\nBe ...
7,Van Morrison,I Forgot That Love Existed,I forgot that love existed troubled in my mind...
8,Andy Williams,Spooky,In the cool of the evening when everything is ...
9,Michael W. Smith,Love Crusade,"Brother, haven't you noticed \r\nThere's a wa..."


In [36]:
df['text'][0]


"He watches witches weaving wonders  \r\nSeeing the future past and present  \r\nOf messages the universe sends  \r\nInside their crystal balls of thunder  \r\nThey summon through the sphere  \r\nA child so fair so virgin pure  \r\nThey summon me my slave  \r\nBring her to their lure  \r\nThe beauty and the beast  \r\nThe beauty and the beast  \r\nBut I can see the blood on your sins  \r\nShe calls christ in churches crimson  \r\nI've got to take her - Leave her - Take her  \r\nTell me the price if I deliver  \r\nBy one a slave of flesh  \r\nIn vain wasting their breath  \r\nThey guide me through the lake  \r\nThey summon his return  \r\nThe beauty and the beast  \r\nThe beauty and the beast  \r\nThey cast countless conjurations  \r\nHe didn't sign the pact the fact is  \r\nTheir screams of failure fade to whispers  \r\nTrying to heal their mental blisters  \r\nA child so fair so golden pure  \r\nThey didn't realise  \r\nThey summon through the sphere  \r\nThis was the demon's cure  \r

In [37]:
df.shape


(5000, 3)

In [38]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)


In [39]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [ ]:
df['text'] = df['text'].apply(lambda x: tokenization(x))


In [41]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [43]:
similarity[0]


array([1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       1.17830557e-02, 7.64076856e-04, 1.56537209e-02])

In [44]:
df[df['song'] == 'Crying Over You']


,artist,song,text


In [48]:
def recommendation(song_df):
    # Check if the song exists in the DataFrame
    if song_df in df['song'].values:
        idx = df[df['song'] == song_df].index[0]
        distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

        songs = []
        for m_id in distances[1:21]:
            songs.append(df.iloc[m_id[0]].song)
        
        return songs
    else:
        return ["Song not found in the database."]

In [49]:
recommendation('Crying Over You')


['Song not found in the database.']

In [50]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))